In [48]:
from elasticsearch import Elasticsearch, NotFoundError

es = Elasticsearch([{'host':'localhost','port':9200}])
es

idx = 'redditors.followers.correlation'
try:
    es.indices.create(idx)
except:
    pass

In [49]:
results = es.search(scroll='360m', body =       
{
  "query": {
    "bool": {
      "must": [
        {
          "exists": {
            "field": "commenters_sampled"
          }
        }
      ]
    }
  }
}, index = 'redditors', size = 10000)

In [60]:
for result in results['hits']['hits']:
    commenters = result['_source']['commenters']
    doc = None
    for commenter in commenters:
        try:
            commenter_doc = es.get(index = 'redditors.followers.sample', id = commenter)
            
            try:
                bm = commenter_doc['_source']['bm']['overall']
            except KeyError:
                continue
            
            if doc is None:
                doc = {
                    "name": result['_id'],
                    "bm": result['_source']['bm'],
                    "followers_bm": {
                        "total": 0,
                        "types": {
                            "ISTJ": 0, "ISFJ": 0, "IIFJ": 0, "IITJ": 0,
                            "ISTP": 0, "ISFP": 0, "IIFP": 0, "IITP": 0,
                            "ESTP": 0, "ESFP": 0, "EIFP": 0, "EITP": 0,
                            "ESTJ": 0, "ESFJ": 0, "EIFJ": 0, "EITJ": 0
                        },
                        "E": {
                            "count": 0,
                            "average": 0.0
                        },
                        "I": {
                            "count": 0,
                            "average": 0.0
                        },
                        "S": {
                            "count": 0,
                            "average": 0.0
                        },
                        "N": {
                            "count": 0,
                            "average": 0.0
                        },
                        "T": {
                            "count": 0,
                            "average": 0.0
                        },
                        "F": {
                            "count": 0,
                            "average": 0.0
                        },
                        "J": {
                            "count": 0,
                            "average": 0.0
                        },
                        "P": {
                            "count": 0,
                            "average": 0.0
                        }
                    } 
                }
            
            doc['followers_bm']['total'] = doc['followers_bm']['total'] + 1
            
            doc['followers_bm']['types'][bm['type']] = doc['followers_bm']['types'][bm['type']] + 1
            
            if 'Feeling' in bm.keys():
                doc['followers_bm']['F']['count'] = doc['followers_bm']['F']['count'] + 1
                doc['followers_bm']['F']['average'] = doc['followers_bm']['F']['average'] + bm['Feeling']
            if 'Intuitive' in bm.keys():
                doc['followers_bm']['N']['count'] = doc['followers_bm']['N']['count'] + 1
                doc['followers_bm']['N']['average'] = doc['followers_bm']['N']['average'] + bm['Intuitive']
            if 'Perceiving' in bm.keys():
                doc['followers_bm']['P']['count'] = doc['followers_bm']['P']['count'] + 1
                doc['followers_bm']['P']['average'] = doc['followers_bm']['P']['average'] + bm['Perceiving']
            if 'Introvert' in bm.keys():
                doc['followers_bm']['I']['count'] = doc['followers_bm']['I']['count'] + 1
                doc['followers_bm']['I']['average'] = doc['followers_bm']['I']['average'] + bm['Introvert']
                
            if 'Extrovert' in bm.keys():
                doc['followers_bm']['E']['count'] = doc['followers_bm']['E']['count'] + 1
                doc['followers_bm']['E']['average'] = doc['followers_bm']['E']['average'] + bm['Extrovert']
            if 'Thinking' in bm.keys():
                doc['followers_bm']['T']['count'] = doc['followers_bm']['T']['count'] + 1
                doc['followers_bm']['T']['average'] = doc['followers_bm']['T']['average'] + bm['Thinking']
            if 'Judging' in bm.keys():
                doc['followers_bm']['J']['count'] = doc['followers_bm']['J']['count'] + 1
                doc['followers_bm']['J']['average'] = doc['followers_bm']['J']['average'] + bm['Judging']
            if 'Sensing' in bm.keys():
                doc['followers_bm']['S']['count'] = doc['followers_bm']['S']['count'] + 1
                doc['followers_bm']['S']['average'] = doc['followers_bm']['S']['average'] + bm['Sensing']
        except NotFoundError:
            pass
        
    if doc['followers_bm']['F']['count'] > 0:
        doc['followers_bm']['F']['average'] = doc['followers_bm']['F']['average'] / doc['followers_bm']['F']['count']
    
    if doc['followers_bm']['N']['count'] > 0:
        doc['followers_bm']['N']['average'] = doc['followers_bm']['N']['average'] / doc['followers_bm']['N']['count']
        
    if doc['followers_bm']['P']['count'] > 0:
        doc['followers_bm']['P']['average'] = doc['followers_bm']['P']['average'] / doc['followers_bm']['P']['count']
        
    if doc['followers_bm']['I']['count'] > 0:
        doc['followers_bm']['I']['average'] = doc['followers_bm']['I']['average'] / doc['followers_bm']['I']['count']
        
    if doc['followers_bm']['E']['count'] > 0:
        doc['followers_bm']['E']['average'] = doc['followers_bm']['E']['average'] / doc['followers_bm']['E']['count']
        
    if doc['followers_bm']['T']['count'] > 0:
        doc['followers_bm']['T']['average'] = doc['followers_bm']['T']['average'] / doc['followers_bm']['T']['count']
        
    if doc['followers_bm']['J']['count'] > 0:
        doc['followers_bm']['J']['average'] = doc['followers_bm']['J']['average'] / doc['followers_bm']['J']['count']
        
    if doc['followers_bm']['S']['count'] > 0:
        doc['followers_bm']['S']['average'] = doc['followers_bm']['S']['average'] / doc['followers_bm']['S']['count']
    
    for key in doc['followers_bm']['types'].keys():
        doc['followers_bm']['types'][key] = (doc['followers_bm']['types'][key] / doc['followers_bm']['total']) * 100.0
    
    for key in doc['followers_bm'].keys():
        entry = doc['followers_bm'][key]
        
        if isinstance(entry, dict) and 'count' in entry.keys():
            entry['percentage'] = entry['count'] / doc['followers_bm']['total']
    
    es.index(index = idx, id = result['_id'], body = doc)
